## Text Mining 

<li> Include some few lines explaining your pipeline strategy at the beginning.



# Imports 

In [2]:
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from concurrent.futures import ThreadPoolExecutor
import statsmodels.api as sm
import numpy as np 


# Headers
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
 
# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

# Functions

In [7]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    

    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            ck = True
            

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

# Opening Booking browser

In [8]:
# Opens the Booking.com website 

dfolder='/Users/nataliabeltran/Downloads/BSE_courses/Term_2_courses/22DM014_Introduction_to_Text_Mining_NLP/ta_session_text_mining/booking_assignment'
geko_path='/Users/nataliabeltran/Downloads/geckodriver'
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

# Reject the cookies 

In [9]:
# Rejects the cookies after logging into the Booking website
x_path_cookies = '//button[@id="onetrust-reject-all-handler"]'
check_and_click(browser, x_path_cookies, 'xpath')

# City Selection

Click on City to travel to 

In [10]:
# Opens the city to travel section
browser.find_element(by='xpath',value='//input[@id=":re:"]').click()

Input city names 

In [11]:
# allows us to input a city to scrape hotels for 
place = input('Where do you want to go?')
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

# Dates

Select dates

In [12]:
# Opens the calendar section 
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

browser.find_element('css selector',css).click()

Dates table

In [13]:
# Gets all the dates of the calendar
path='//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'

dates = browser.find_elements('xpath',path)


for date in dates:
    print(date.get_attribute("data-date"))

2024-01-25
2024-01-26
2024-01-27
2024-01-28
2024-01-29
2024-01-30
2024-01-31
2024-02-01
2024-02-02
2024-02-03
2024-02-04
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-10
2024-02-11
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-17
2024-02-18
2024-02-19
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-24
2024-02-25
2024-02-26
2024-02-27
2024-02-28
2024-02-29


<p> Select the desired dates 
<p> Looking at Barcelona & Lisbon (12-16 for treatment week, 19-23 control week ). Write more information 


Create function from this so its easier to call and run

In [14]:
def click_element(browser, xpath, num_clicks=1):
    """
    Clicks an element identified by the given XPath multiple times.

    Parameters:
    - browser (WebDriver): The browser instance to interact with.
    - xpath (str): XPath of the element to click.
    - num_clicks (int): Number of times to click the element. Default is 1.
    """
    
    for _ in range(num_clicks):
        browser.find_element('xpath', xpath).click()

def select_month(browser, xpath_button, num_month=1):
    """
    Selects a month by clicking an element identified by the given XPath.

    Parameters:
    - browser (WebDriver): The browser instance to interact with.
    - xpath_button (str): XPath of the button to select the month.
    - num_month (int): Number of months from the current month to navigate to get to desired month. 
    
    Notes: 
        - For num_month the default is 1, change depending on what month you are looking for. 
    """

    for _ in range(num_month):
        browser.find_element('xpath', xpath_button).click()

def select_date_range(browser, from_day, to_day):
    """
    Selects a date range on a calendar.

    Parameters:
    - browser (WebDriver): The browser instance to interact with.
    - from_day (str): The start day of the date range in 'DD' format.
    - to_day (str): The end day of the date range in 'DD' format.
    """

    path = '//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'
    dates = browser.find_elements('xpath', path)

    for date in dates:
        date_value = date.get_attribute("data-date")
        if date_value == f"2024-06-{from_day}":
            date.click()
        if date_value == f"2024-06-{to_day}":
            date.click()
            break

def calendar_days(browser, from_day='19', to_day='23'):
    """
    Main function to perform a series of actions on a webpage.

    Parameters:
    - browser (WebDriver): The browser instance to interact with.
    - from_day (str): The start day of the date range in 'DD' format. Default is '19'.
    - to_day (str): The end day of the date range in 'DD' format. Default is '23'.
    """

    num_clicks = 1
    arrow_xpath = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button/span/span'
    click_element(browser, arrow_xpath, num_clicks)

    num_month = 3 
    month_xpath = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[2]'
    select_month(browser, month_xpath, num_month)

    select_date_range(browser, from_day, to_day)

"""
Example:
main(browser, from_day='15', to_day='20')  # Pass the browser object and custom from_day/to_day values
"""


"\nExample:\nmain(browser, from_day='15', to_day='20')  # Pass the browser object and custom from_day/to_day values\n"

In [15]:
# Run the main function
calendar_days(browser, '19', '23') 

Search

In [16]:
#XPath to search 
my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'

check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/E

<p>Scrape date, room price, hotel name and hotel description.

Pages

In [17]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//div[@class="ab95b25344"]')
    return a[0].text.split("\n")[-1]
    #return(int(a[-1].text))


pages = get_number_pages(browser)

print(pages)


#def next_page_selector(browser):
    #next_page_button = browser.find_elements('xpath', '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 a16ddf9c57"]').click()



40


XPaths

In [18]:
# PAGES XPATH 
css_pages = 'div.b16a89683f:nth-child(3) > button:nth-child(1) > span:nth-child(1) > span:nth-child(1)'

#XPATHS
sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
hotels_xpath = './/div[@class="f6431b446c a15b38c233"]'
ratings_xpath = './/div[@class="a3b8729ab1 d86cee9b25"]'
prices_xpath = './/span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]'



# EXTRACT HOTEL NAMES, RATINGS, & PRICE

In [19]:
def hotel_information_scraping(browser, hotels_xpath, ratings_xpath, prices_xpath, css_pages, pages): 
    """ 
    Scrapes hotel information from a series of pages using Selenium. 

    Parameters: 
        - browser (Webdriver): The Selenium Webdriver instance. 
        - hotels_xpath (str): XPath to locate the hotel name element within each section of hotel options. 
        - ratings_xpath (str): XPath to locate the rating element within each section of hotel options. 
        - prices_xpath (str): XPath to locate the price element within each section of hotel options. 
        - css_pages (str): CSS Selector to locate the element for switiching pages. 
        - pages (int): The total number of pages to scrape found using function get_number_pages(). 

    Returns: 
        - pd.Dataframe: A Dataframe containing the scraped hotel information. 

    Notes: 
        - The function prints the hotel information for each section on each page. 
        - If using for a different website, the xpath for the Section & Hotel URLs need to be replaced with the correct path. 

    Example: 
       '''python:  df_hotel = scrape_hotel_data(browser, hotels,xpath, ratings_xpath, prics_xpath, css_pages, pages)'''
    
    """

    # Lists to store the extracted information
    hotels_list = []
    ratings_list = []
    prices_list = []
    url_list = []
    

    for page in range(int(pages)+1): 
        #Print page that it is in 
        print(f'Page: {page + 1}')
        sections = browser.find_elements('xpath', '//div[@class="c066246e13"]')
        for hotel in sections:
            try:
                hotel_name = hotel.find_element('xpath', hotels_xpath).text

            except NoSuchElementException: 
                print("Element could not be found. Printing page source:", browser.page_source)
                continue # Skip to the next iteration if the hotel section was not found 

            except StaleElementReferenceException: 
                    print("StaleElementReferenceException. Retrying...")
                    #Retry finding the element by waiting for it be present 
                    wait = WebDriverWait(browser, 10)

                    try: 
                        hotel_name = wait.until(EC.presence_of_element_located(('xpath', hotels_xpath))).text 
                    except NoSuchElementException: 
                        print("Element still cannot be found. Skipping to next Hotel.")
                        continue
                
            try:
                rating = hotel.find_element('xpath', ratings_xpath).text
            except:
                rating = "N/A"

            try:
                price = hotel.find_element('xpath', prices_xpath).text
            except:
                price = "N/A"  # I don't think i need an N/A for the price cause there should always be a price 

            try: 
                url = hotel.find_element('xpath', './/a[@href]')
                hotel_url= url.get_attribute('href')
            except:
                hotel_url = "N/A"
        
            # Print the hotel information as it runs through each section
            print("Hotel Name:", hotel_name)
            print("Rating:", rating)
            print("Price", price)
            print("Hotel URL", hotel_url)
            print("\n")

            # Append the information to their respective lists
            hotels_list.append(hotel_name)
            ratings_list.append(rating)
            prices_list.append(price)
            url_list.append(hotel_url)



            # Switch page with CSS Selector
        switch_pages = browser.find_element('css selector', css_pages).click()
        time.sleep(2)


    # Create a dictonary to store all list together
    dict = { 'Hotel Name': hotels_list, 'Rating': ratings_list, 'Price': prices_list, 'Hotel URL': url_list }

    # Create a dataframe with the dictionary
    df_lisbc = pd.DataFrame(dict)
    return df_lisbc



In [21]:
# Running hotel scrape function. Repeat for all desired cities needed to scrape. 
df_lisbc= hotel_information_scraping(browser, hotels_xpath, ratings_xpath, prices_xpath, css_pages, pages)

Page: 1
Hotel Name: Succeed Campo Pequeno Suites
Rating: 7,6
Price € 380
Hotel URL https://www.booking.com/hotel/pt/succeed-campo-pequeno-suites-lisboa.es.html?label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AtX1ya0GwAIB0gIkMTRmZjUxZGEtYWNhMi00MTFjLThjZDItNWVmMDI1NzU2NGNm2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-19&checkout=2024-06-23&dest_id=-2167973&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=0ff46b3854a502da&srepoch=1706195744&all_sr_blocks=1051493104_378159955_0_0_0&highlighted_blocks=1051493104_378159955_0_0_0&matching_block_id=1051493104_378159955_0_0_0&sr_pri_blocks=1051493104_378159955_0_0_0__38000&from=searchresults#hotelTmpl


Hotel Name: HF Fenix Urban
Rating: 8,2
Price € 452
Hotel URL https://www.booking.com/hotel/pt/hf-fenix-urban.es.html?label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AtX1ya0GwAIB0gIkMTRmZjUxZGEtYWNhMi00MTFjLThjZD

<p> BREAKDOWN OF THE DIFFERENT DATAFRAMES 
<p> df_libt is treatment week for Lisbon (12-16)
<p> df_libc is control week for Lisbon (19-23)
<p> df_bart is treatment week for Barcelona (12-16)
<p> df_barc is control week for Barcelona (19-23)

# Extract Description

add docstrings to function

In [22]:
def description_info(url):

    """
    Extracts and returns the description text from a specified URL. 
    
    Parameters: 
        - url (str): The URL of the webpage to extract the description text from. 

    Returns: 
        - str or None: The extracted description text that is found, otherwise a None. 

    Raises: 
        - request.exceptions.RequestException: In place in case there is a potential issue with the HTTP request.
    """
    
    try:
        req = requests.get(url, headers=headers, timeout=60)
        req.raise_for_status() 
    except requests.exceptions.RequestException as e:
        print(f"Can't process the {url}: {e}")
        return None

    soup = BeautifulSoup(req.text, 'html.parser')
    description_text = soup.find('p', class_='a53cbfa6de b3efd73f69')

    if description_text:
        return description_text.get_text(strip=True)
    else:
        print("Description could not be found")
        return None



In [27]:
# Create an empty list to store the descriptions
descriptions_list = []

# Iterate through the URLs and scrape descriptions sequentially
for url in df_lisbc['Hotel URL']:
    description = description_info(url)
    descriptions_list.append(description)

# Assign the descriptions to the 'Descriptions' column in the DataFrame
df_lisbc['Descriptions'] = descriptions_list

Can't process the https://www.booking.com/hotel/pt/alfama-lisbon-lounge-suites.es.html?label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AtX1ya0GwAIB0gIkMTRmZjUxZGEtYWNhMi00MTFjLThjZDItNWVmMDI1NzU2NGNm2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-19&checkout=2024-06-23&dest_id=-2167973&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=22&hapos=697&sr_order=popularity&srpvid=0ff46b3854a502da&srepoch=1706195888&all_sr_blocks=44040302_88538307_0_2_0&highlighted_blocks=44040302_88538307_0_2_0&matching_block_id=44040302_88538307_0_2_0&sr_pri_blocks=44040302_88538307_0_2_0__77040&from=searchresults#hotelTmpl: HTTPSConnectionPool(host='www.booking.com', port=443): Read timed out.


## Create Dummy Variables

Barcelona 13-15 June

In [97]:
# Dummmy Variables Barcelona during 13-15 June 

df_bart['Treatment_Period'] = 1
df_bart['Treatment_City'] = 1 

csv = df_bart.to_csv('Barcelona_Treatment_12_16.csv')

Barcelona 20-22 June

In [126]:
# Dummy Variable Barcelona during 20-22 June 
df_barc['Treatment_Period'] = 0
df_barc['Treatment_City'] = 1 

csv = df_barc.to_csv('Barcelona_Control_19_23.csv')

Lisbon 13-15 June 

In [53]:
# Dummy Variable Madrid duing 13-15 June 
df_lisbt['Treatment_Period'] = 1
df_lisbt['Treatment_City'] = 0

csv = df_lisbt.to_csv('Lisbon_Treatment_12_16.csv')

Lisbon 20-22 June

In [73]:
# Dummy Variable Madrid during 20-22 June 
df_lisbc['Treatment_Period'] = 0
df_lisbc['Treatment_City'] = 0

csv = df_lisbc.to_csv('Lisbon_Control_19_23.csv')

# Import all the CSV as Dataframes

In [128]:
#Barcelona Dataframes
df_bart = pd.read_csv("Barcelona_Treatment_12_16.csv")
df_barc = pd.read_csv("Barcelona_Control_19_23.csv")

# Lisbon Dataframes
df_lisbt = pd.read_csv("Lisbon_Treatment_12_16.csv")
df_lisbc = pd.read_csv("Lisbon_Control_19_23.csv")

# Concatinate all the DataFrames

In [132]:
df_total = pd.concat([df_bart, df_barc, df_lisbt ,df_lisbc], ignore_index=True)

In [134]:
df_total.tail()

,Unnamed: 0,Hotel Name,Rating,Price,Hotel URL,Descriptions,Treatment_Period,Treatment_City
4099,1021,Casa Santa Marinha,"8,3",€ 704,https://www.booking.com/hotel/pt/ana-apartamen...,"La Casa Santa Marinha se encuentra en Lisboa, ...",0,0
4100,1022,Hotel Santa Justa,"9,1",€ 1.182,https://www.booking.com/hotel/pt/santa-justa.e...,"El Santa Justa Hotel, de 4 estrellas, goza de ...",0,0
4101,1023,The House on the Pink Street in Lisbon,"9,7",€ 1.012,https://www.booking.com/hotel/pt/the-house-on-...,The House on the Pink Street in Lisbon se encu...,0,0
4102,1024,Cais Sodré Aptm with Amazing View,"9,0",€ 843,https://www.booking.com/hotel/pt/26-travessa-r...,Cais Sodré Aptm with Amazing View ofrece wifi ...,0,0
4103,1025,FLH Chiado Stylish Flat,NaN,€ 810,https://www.booking.com/hotel/pt/flh-chiado-st...,FLH Chiado Stylish Flat ofrece wifi gratis y e...,0,0


# Preprocess Dataframe 

Removing URL links from Dataframe 

In [135]:
column_to_remove = ['Hotel URL']

df_total = df_total.drop(columns=column_to_remove)

df_total

Remove the € from the Price column 

In [139]:
# Remove the € sign from the price column 
df_total['Price'] = df_total["Price"].str.replace('€', '')

# Drop rows with NaNs potentially? 

Create the interaction between the treatment period and treatment citty 

In [142]:
df_total['Interaction_Period_City'] = df_total['Treatment_Period']  * df_total['Treatment_City']
df_total

,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City,Interaction_Period_City
0,0,Casa Abamita,"8,5",987,Los Apartamentos Mayerling Abamita se encuentr...,1,1,1
1,1,Hotel Barcelona Condal Mar Affiliated by Meliá,"8,0",1.011,"El Hotel Barcelona Condal Mar, un hotel modern...",1,1,1
2,2,Catalonia Sagrada Familia,"8,2",673,El Catalonia Sagrada Familia se halla a 15 min...,1,1,1
3,3,Acta CITY47,"8,5",726,"El Acta CITY47 está situado en Barcelona, a 5 ...",1,1,1
4,4,Center Gran Via,"8,7",663,El Hotel Center Gran Via está en el centro de ...,1,1,1
...,...,...,...,...,...,...,...,...
4099,1021,Casa Santa Marinha,"8,3",704,"La Casa Santa Marinha se encuentra en Lisboa, ...",0,0,0
4100,1022,Hotel Santa Justa,"9,1",1.182,"El Santa Justa Hotel, de 4 estrellas, goza de ...",0,0,0
4101,1023,The House on the Pink Street in Lisbon,"9,7",1.012,The House on the Pink Street in Lisbon se encu...,0,0,0
4102,1024,Cais Sodré Aptm with Amazing View,"9,0",843,Cais Sodré Aptm with Amazing View ofrece wifi ...,0,0,0


Convert columns from strings to numerical values

In [152]:
#Original column conversion to numerical
df_total['Price'] = pd.to_numeric(df_total['Price'])

# Dummy Variables conversion to numerical
df_total['Treatment_Period'] = pd.to_numeric(df_total['Treatment_Period'])
df_total['Treatment_City'] = pd.to_numeric(df_total['Treatment_City'])
df_total['Interaction_Period_City'] = pd.to_numeric(df_total['Interaction_Period_City'])

# Exploratory Data Analysis ???

In [ ]:
# Create some graphs with the price by city to better understand the variable (ask Dani)
# Look at average price of each of the 4 treatment/control groups 
# Potentially graph price against each of the treatments 

# OLS Regression

OLS Regression for treatment period

In [155]:
# Regression for Treatment Period 
const_period= sm.add_constant(df_total['Treatment_Period'])
model_per = sm.OLS(df_total['Price'], const_period)
ols_period = model_per.fit()
print(ols_period.summary())



                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     14.25
Date:                Thu, 25 Jan 2024   Prob (F-statistic):           0.000162
Time:                        12:10:56   Log-Likelihood:                -29923.
No. Observations:                4104   AIC:                         5.985e+04
Df Residuals:                    4102   BIC:                         5.986e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              328.3749      7.841  

OLS Regression for treatment city 

In [156]:
# Regression for Treatment Period 
const_city= sm.add_constant(df_total['Treatment_City'])
model_city = sm.OLS(df_total['Price'], const_city)
ols_city = model_city.fit()
print(ols_city.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     703.3
Date:                Thu, 25 Jan 2024   Prob (F-statistic):          3.63e-143
Time:                        12:12:47   Log-Likelihood:                -29606.
No. Observations:                4104   AIC:                         5.922e+04
Df Residuals:                    4102   BIC:                         5.923e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            485.3830      7.257     66.

OLS Regression for interaction of treatment period and city 

In [157]:
# Regression for Treatment Period 
const_interaction= sm.add_constant(df_total['Interaction_Period_City'])
model_interaction = sm.OLS(df_total['Price'], const_interaction)
ols_interaction = model_interaction.fit()
print(ols_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     61.14
Date:                Thu, 25 Jan 2024   Prob (F-statistic):           6.71e-15
Time:                        12:12:49   Log-Likelihood:                -29900.
No. Observations:                4104   AIC:                         5.980e+04
Df Residuals:                    4102   BIC:                         5.982e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

# REG EX 